<a href="https://colab.research.google.com/github/victornator25/CODIGOS-GV/blob/main/ActualizarW%26W_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este código actualiza el sistema win and win, es importante que el nombre de las columnas coincida, actualizar los índices de hoja de ser necesario y no editar columnas y encabezados o marcará error.
No se debe mover el orden de las hojas en el win, de lo contrario no funcionará como debe.

In [ ]:
#CODIGO QUE ACTUALIZA LA ORDEN DE TRABAJO
"""
-Es importante ver cual es el número de lote a actualizar en el win and win
(las que no tienen orden de trabajo son las nuevas)

-En las google sheets tambien se puede eliminar duplicados de ser necesario(si el batch fue más grande del requerido)

-Al final de la actualización colorear de azul para que sea más facil identificar

-No cambiar el nombre de las hojas o también modificarlas en el código

-Solo restaría actualizar el ITEM en cada hoja de win and win_2 (código pendiente)

-SI VAS A HACER PRUEBAS HAZLO EN LAS COPIAS
"""
######### ACTUALIZAR A NECESIDADES ########
dim_lote_actualizar=3
###########################################

doc_solicitud='W&W SOLICITUDES GV'  #HOJA PARA LEER
doc_destino  ='win and win_2'  #HOJA A MODIFICAR

#LA SIGUIETE SECCIÓN DE CÓDIGO ACTUALIZA LA ORDEN DE TRABAJO DE LAS SOLICITUDES

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

import pandas as pd

gc = gspread.authorize(creds)

# Abre la hoja de cálculo
spreadsheet2 = gc.open(doc_solicitud)
worksheet2 = spreadsheet2.get_worksheet(0)  # Supongamos que la hoja de interés está en la posición 0

# Obtén los datos de la hoja de cálculo
data2 = worksheet2.get_all_values()

# Crea el DataFrame SOLICITUDES_WandW
columnas_solicitudes = ['Marca temporal', 'FECHA SOLICITUD', 'NOMBRE SOLICITANTE', 'ACTIVIDAD', 'PRODUCTO', 'SERVICIO',
                        'CLIENTE', 'RESPONSABLE', 'ORDEN DE PRODUCCION', 'COMENTARIOS DEL SOLICITANTE', 'PRIORIDAD',
                        'FECHA DE REQUERIMIENTO', 'ORDEN DE TRABAJO']
SOLICITUDES_WandW = pd.DataFrame(data2[1:], columns=columnas_solicitudes)

# Definir una función para generar el valor de 'OT' según la lógica dada
def generar_ot(fila):
    # Obtener las iniciales y el número de fila ajustado
    inicial_solicitante = fila['NOMBRE SOLICITANTE'][0]
    if ' ' in fila['NOMBRE SOLICITANTE']:
        segunda_inicial_solicitante = fila['NOMBRE SOLICITANTE'].split(' ')[1][0]
    else:
        segunda_inicial_solicitante = ''

    inicial_encargado = fila['RESPONSABLE'][0]
    if ' ' in fila['RESPONSABLE']:
        segunda_inicial_encargado = fila['RESPONSABLE'].split(' ')[1][0]
    else:
        segunda_inicial_encargado = ''

    numero_fila_ajustado = int(fila.name) - 41

    # Concatenar las partes para formar 'OT'
    ot = f"{inicial_solicitante}{segunda_inicial_solicitante}{inicial_encargado}{segunda_inicial_encargado}{numero_fila_ajustado}"

    return ot

# Aplicar la función a todas las filas
SOLICITUDES_WandW['ORDEN DE TRABAJO'] = SOLICITUDES_WandW.apply(generar_ot, axis=1)

# Filtrar las filas que no tienen valores numéricos en 'OT'
filas_sin_ot_numerico = SOLICITUDES_WandW[SOLICITUDES_WandW['ORDEN DE TRABAJO'].str.isnumeric() == False]

# Obtener las coordenadas de las celdas correspondientes en la hoja de cálculo
filas_sin_ot_numerico_indices = filas_sin_ot_numerico.index + 2  # Ajustar al índice de la hoja de cálculo
celdas_a_actualizar = ['K' + str(indice) for indice in filas_sin_ot_numerico_indices]
"""
# Actualizar la hoja de cálculo solo para las celdas necesarias
for celda, valor in zip(celdas_a_actualizar, filas_sin_ot_numerico['OT']):
    worksheet2.update(celda, valor)

print("Hoja de cálculo actualizada con éxito.")

"""

# Obtener el índice de la columna 'OT'
indice_ot = SOLICITUDES_WandW.columns.get_loc('ORDEN DE TRABAJO')

# Filtrar las filas que no tienen valores numéricos en 'OT'
filas_sin_ot_numerico = SOLICITUDES_WandW[SOLICITUDES_WandW['ORDEN DE TRABAJO'].str.isnumeric() == False]

# Obtener las coordenadas de las celdas correspondientes en la hoja de cálculo
filas_sin_ot_numerico_indices = filas_sin_ot_numerico.index + 2  # Ajustar al índice de la hoja de cálculo
columna_ot = indice_ot + 1  # Ajustar al índice de la hoja de cálculo
celdas_a_actualizar = [f"{chr(ord('A') + columna_ot - 1)}{indice}" for indice in filas_sin_ot_numerico_indices]

# Obtener los valores de 'OT' que se van a actualizar
nuevos_valores_ot = list(filas_sin_ot_numerico['ORDEN DE TRABAJO'])

# Crear una lista de tuplas con las coordenadas y los nuevos valores
actualizaciones = [{'range': celda, 'values': [[valor]]} for celda, valor in zip(celdas_a_actualizar, nuevos_valores_ot)]

# Actualizar la hoja de cálculo solo para las celdas necesarias
worksheet2.batch_update(actualizaciones)

print("Hoja de cálculo actualizada con éxito.")


Hoja de cálculo actualizada con éxito.


In [ ]:
#CODIGO QUE ENVIA SOLICITUDES DEL WIN A LAS RESPECTIVAS HOJAS

#SI POR ERROR SE CORRE DOS VECES SEGUIDAS, CHECAR DUPLICADOS EN win and win_2

# Leer los últimos registros de 'Copia de W&W SOLICITUDES GV'
ultimos_registros = SOLICITUDES_WandW.tail(dim_lote_actualizar)

# Definir función para determinar la hoja en 'Copia de win and win_2' según el encargado
def determinar_hoja(encargado):
    if encargado == 'MARTIN O':
        return doc_destino, 0 #se va al índice de la google sheet del win and win_2
    elif encargado == 'CLAUDIA R':
        return doc_destino, 1
    elif encargado == 'ENRIQUE R':
        return doc_destino, 2
    elif encargado == 'BERTA A':
        return doc_destino, 3
    elif encargado == 'COTIZACIONE S':
        return doc_destino, 4

### INDICE 5 PARA VENDEDORAS
    elif encargado == 'LAURA R':
        return doc_destino, 5
    elif encargado == 'VIRIDIANA A':
        return doc_destino, 5
#   elif encargado == 'LIZBETH L':
#      return doc_destino, 5,7
    elif encargado == 'TATIANA E':
        return doc_destino, 5

    elif encargado == 'NALLELY S':
        return doc_destino, 6
    elif encargado == 'LIZBETH L':
        return doc_destino, 7
    elif encargado == 'JULIO S':
        return doc_destino, 8
    elif encargado == 'FABIAN F':
        return doc_destino, 9
    elif encargado == 'CARLOS H':
        return doc_destino, 10
    elif encargado == 'JORGE C':
        return doc_destino, 11
    elif encargado == 'RAFA V':
        return doc_destino, 12
    elif encargado == 'BENJAMIN F':
        return doc_destino, 13
    elif encargado == 'NESTOR C':
        return doc_destino, 14
    elif encargado == 'AXEL S':
        return doc_destino, 15
    elif encargado == 'IVAN M':
        return doc_destino, 16
    elif encargado == 'ALEX R':
        return doc_destino, 17
    elif encargado == 'VALERIA C':
        return doc_destino, 18
    else:
        # este es para el caso que no sea ninguno de los anteriores
        return doc_destino, 19

# Iterar sobre los registros y enviar cada fila a la hoja correspondiente
for indice, fila in ultimos_registros.iterrows():
    encargado = fila['RESPONSABLE']
    hoja_destino, indice_hoja = determinar_hoja(encargado)

    if hoja_destino is not None:
        # Obtener los valores desde la columna 2 hasta la columna 13
        valores_a_actualizar = fila.iloc[0:13].tolist()
        # Agregar la fila a la hoja de destino en 'Copia de win and win_2' desde la columna 2
        gc.open(hoja_destino).get_worksheet(indice_hoja).append_row(values=[None] + valores_a_actualizar)

        print(f"Fila agregada en {hoja_destino} con éxito.")
    else:
        print(f"No se pudo determinar la hoja destino para la fila {indice}.")




Fila agregada en win and win_2 con éxito.
Fila agregada en win and win_2 con éxito.
Fila agregada en win and win_2 con éxito.
